# About project

### We need to build a classification model based on the limited dataset for propensity to the product "**Consumer Credit**".
<br>

***Datasets***:<br>
`Clients` -- Clients<br>
`Transactions` -- Transactions<br>
<br>

***Description***:<br>

**Clients**<br>
`client_id` - Client ID<br>
`age` - Age<br>
`gender_code` - Gender<br>
`directory` - City<br>
`aMRG_eop` - Mortgage debt balance<br>
`aCSH_eop` - Consumer loan debt balance<br>
`aCRD_eop` - Credit card debt balance<br>
`pCUR_eop` - Current account balance<br>
`pCRD_eop` - Card account balance<br>
`pSAV_eop` - Savings account balance<br>
`pDEP_eop` - Deposit account balance<br>
`sWork_S` - Monthly salary<br>
`tPOS_S` - POS transaction amount<br>
<br>
**Transactions**<br>
`client_id` - Client ID<br>
`TRANSACTION_DT` - Transaction date<br>
`MCC_KIND_CD` - MCC type<br>
`MCC_CD` - MCC code<br>
`CARD_AMOUNT_EQV_CBR` - Transaction amount in Russian rubles<br>

# Data loading

In [15]:
import gdown

def download_from_google(id, save_as):
    gdown.download(f"https://drive.google.com/uc?id={id}", output=save_as, quiet=True)

In [16]:
download_from_google('1NSNd1GGbplxC36IJ6hfXvvo398jz-knV', '/content/Transactions.csv')
download_from_google('1KS3NIupNHli6rnj7zz5YgnEZCbf7Okee', '/content/Clients.csv')

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

# reading datas
clients = pd.read_csv("/content/Clients.csv", delimiter=';')
transactions = pd.read_csv('/content/Transactions.csv', delimiter=';')

# Data preprocessing

## Clients

### Clean data

In [18]:
# Convert age to int
clients['age'] = clients['age'].astype(int)

# Replace commas with periods and convert to float
clients['aMRG_eop'] = clients['aMRG_eop'].str.replace(',', '.').astype(float)
clients['aCSH_eop'] = clients['aCSH_eop'].str.replace(',', '.').astype(float)

clients['aCRD_eop'] = clients['aCRD_eop'].str.replace(',', '.').astype(float)
clients['pCUR_eop'] = clients['pCUR_eop'].str.replace(',', '.').astype(float)

clients['pCRD_eop'] = clients['pCRD_eop'].str.replace(',', '.').astype(float)
clients['pSAV_eop'] = clients['pSAV_eop'].str.replace(',', '.').astype(float)

clients['pDEP_eop'] = clients['pDEP_eop'].str.replace(',', '.').astype(float)
clients['sWork_S'] = clients['sWork_S'].str.replace(',', '.').astype(float)
clients['tPOS_S'] = clients['tPOS_S'].str.replace(',', '.').astype(float)

In [19]:
num_obscl = clients.shape[0]
print(f"Number of Observations: {num_obscl}")
clients

Number of Observations: 1000


,client_id,age,gender_code,directory,aMRG_eop,aCSH_eop,aCRD_eop,pCUR_eop,pCRD_eop,pSAV_eop,pDEP_eop,sWork_S,tPOS_S
0,1,38,Ж,Санкт-Петербург,0.00,742601.31,0.0,0.00,166353.83,0.00,0.0,286133.38,114454.28
1,2,38,Ж,Нижний Новгород,0.00,250000.00,540.0,0.00,372.19,0.00,0.0,107920.36,92437.27
2,3,37,Ж,Нижний Новгород,0.00,247232.80,0.0,0.00,89735.56,0.00,0.0,88649.15,73698.68
3,4,37,М,Нижний Новгород,0.00,1195448.42,0.0,0.00,314498.64,0.00,0.0,198273.64,133039.22
4,5,58,М,Нижний Новгород,5045174.72,0.00,0.0,0.81,179383.40,0.00,0.0,158434.28,54120.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,48,Ж,Новый Уренгой,0.00,1000000.00,0.0,1500000.00,243169.83,263298.50,0.0,117428.00,90782.05
996,997,51,М,Новый Уренгой,469532.28,0.00,0.0,0.00,70391.45,351.08,0.0,205222.71,83319.15
997,998,49,М,Тюмень,0.00,1204003.78,0.0,45.00,56496.65,0.00,0.0,137305.88,165253.09
998,999,45,Ж,Сургут,0.00,655645.42,0.0,43.45,89718.27,0.00,380000.0,97142.73,216913.16


In [20]:
clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   client_id    1000 non-null   int64  
 1   age          1000 non-null   int64  
 2   gender_code  1000 non-null   object 
 3   directory    1000 non-null   object 
 4   aMRG_eop     1000 non-null   float64
 5   aCSH_eop     1000 non-null   float64
 6   aCRD_eop     1000 non-null   float64
 7   pCUR_eop     1000 non-null   float64
 8   pCRD_eop     1000 non-null   float64
 9   pSAV_eop     1000 non-null   float64
 10  pDEP_eop     1000 non-null   float64
 11  sWork_S      1000 non-null   float64
 12  tPOS_S       1000 non-null   float64
dtypes: float64(9), int64(2), object(2)
memory usage: 101.7+ KB


In [21]:
display(clients)

,client_id,age,gender_code,directory,aMRG_eop,aCSH_eop,aCRD_eop,pCUR_eop,pCRD_eop,pSAV_eop,pDEP_eop,sWork_S,tPOS_S
0,1,38,Ж,Санкт-Петербург,0.00,742601.31,0.0,0.00,166353.83,0.00,0.0,286133.38,114454.28
1,2,38,Ж,Нижний Новгород,0.00,250000.00,540.0,0.00,372.19,0.00,0.0,107920.36,92437.27
2,3,37,Ж,Нижний Новгород,0.00,247232.80,0.0,0.00,89735.56,0.00,0.0,88649.15,73698.68
3,4,37,М,Нижний Новгород,0.00,1195448.42,0.0,0.00,314498.64,0.00,0.0,198273.64,133039.22
4,5,58,М,Нижний Новгород,5045174.72,0.00,0.0,0.81,179383.40,0.00,0.0,158434.28,54120.55
...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,996,48,Ж,Новый Уренгой,0.00,1000000.00,0.0,1500000.00,243169.83,263298.50,0.0,117428.00,90782.05
996,997,51,М,Новый Уренгой,469532.28,0.00,0.0,0.00,70391.45,351.08,0.0,205222.71,83319.15
997,998,49,М,Тюмень,0.00,1204003.78,0.0,45.00,56496.65,0.00,0.0,137305.88,165253.09
998,999,45,Ж,Сургут,0.00,655645.42,0.0,43.45,89718.27,0.00,380000.0,97142.73,216913.16


## Transactions

### Clean data

In [22]:
import pandas as pd

# Convert TRANSACTION_DT to datetime
transactions['TRANSACTION_DT'] = pd.to_datetime(transactions['TRANSACTION_DT'])

# Convert MCC_CD to float
transactions['MCC_CD'] = transactions['MCC_CD'].astype(float)

# Replace commas with periods and convert CARD_AMOUNT_EQV_CBR to float
transactions['CARD_AMOUNT_EQV_CBR'] = transactions['CARD_AMOUNT_EQV_CBR'].str.replace(',', '.').astype(float)


<ipython-input-22-8a9415238e83>:4: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  transactions['TRANSACTION_DT'] = pd.to_datetime(transactions['TRANSACTION_DT'])


In [23]:
num_obstr = transactions.shape[0]
print(f"Number of Observations: {num_obstr}")
transactions

Number of Observations: 77666


,client_id,TRANSACTION_DT,MCC_KIND_CD,MCC_CD,CARD_AMOUNT_EQV_CBR
0,1,2020-01-09,Розничные магазины,5411.0,51.64
1,1,2020-01-09,Различные магазины,5921.0,722.30
2,1,2020-01-09,Розничные магазины,5411.0,769.27
3,1,2020-05-09,Розничные магазины,5411.0,5.90
4,1,2020-05-09,Личные услуги,7230.0,209.00
...,...,...,...,...,...
77661,1000,2020-09-28,Розничные магазины,5411.0,200.96
77662,1000,2020-09-28,Магазины одежды,5651.0,585.00
77663,1000,2020-09-29,Розничные магазины,5331.0,331.17
77664,1000,2020-09-29,Розничные магазины,5411.0,658.00


In [24]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77666 entries, 0 to 77665
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   client_id            77666 non-null  int64         
 1   TRANSACTION_DT       77666 non-null  datetime64[ns]
 2   MCC_KIND_CD          77439 non-null  object        
 3   MCC_CD               77466 non-null  float64       
 4   CARD_AMOUNT_EQV_CBR  77666 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 3.0+ MB


В данной таблице есть пропуски. Отобразим их:

In [25]:
# Finding all the missings
transactions[transactions.isna().any(axis=1)]

,client_id,TRANSACTION_DT,MCC_KIND_CD,MCC_CD,CARD_AMOUNT_EQV_CBR
397,6,2020-09-19,NaN,NaN,1365.36
971,14,2020-02-09,NaN,NaN,59.00
1006,15,2020-01-09,NaN,NaN,3840.66
1083,16,2020-02-09,NaN,NaN,59.00
1213,19,2020-09-28,NaN,NaN,340599.00
...,...,...,...,...,...
76606,986,2020-09-23,NaN,NaN,54000.00
76680,987,2020-09-23,NaN,NaN,225000.00
76894,990,2020-01-09,NaN,742.0,4264.00
76915,990,2020-04-09,NaN,742.0,400.00


In [26]:
# Checking MCC_CD and MCC_KIND_CD
desired_mcc_cd = 742.0
transactions[transactions['MCC_CD'] == desired_mcc_cd]

,client_id,TRANSACTION_DT,MCC_KIND_CD,MCC_CD,CARD_AMOUNT_EQV_CBR
2217,33,2020-02-09,NaN,742.0,2405.0
4243,58,2020-09-14,Контрактные услуги,742.0,3028.0
4530,64,2020-02-09,NaN,742.0,172.0
8208,113,2020-09-22,Контрактные услуги,742.0,2079.0
9193,127,2020-02-09,NaN,742.0,2600.0
9205,127,2020-08-09,NaN,742.0,500.0
11979,162,2020-09-22,Контрактные услуги,742.0,4000.0
13660,185,2020-09-14,Контрактные услуги,742.0,1200.0
13661,185,2020-09-14,Контрактные услуги,742.0,2000.0
13664,185,2020-09-15,Контрактные услуги,742.0,1230.0


In [27]:
# Check for missing values
missing_values = transactions.isnull().sum()
print(missing_values)
# We have only 27 rows which have MCC_CD but not have MCC_KIND_CD, so it is not very significant and will not have much cause on result, so we can just drop it
transactions = transactions.dropna()

client_id                0
TRANSACTION_DT           0
MCC_KIND_CD            227
MCC_CD                 200
CARD_AMOUNT_EQV_CBR      0
dtype: int64


# Data merging

In [28]:
# merging
df_merged = pd.merge(clients, transactions, on='client_id', how='inner')

In [29]:
num_obsdata = df_merged.shape[0]
print(f"Number of Observations: {num_obsdata}")
df_merged.head(5)

Number of Observations: 77439


,client_id,age,gender_code,directory,aMRG_eop,aCSH_eop,aCRD_eop,pCUR_eop,pCRD_eop,pSAV_eop,pDEP_eop,sWork_S,tPOS_S,TRANSACTION_DT,MCC_KIND_CD,MCC_CD,CARD_AMOUNT_EQV_CBR
0,1,38,Ж,Санкт-Петербург,0.0,742601.31,0.0,0.0,166353.83,0.0,0.0,286133.38,114454.28,2020-01-09,Розничные магазины,5411.0,51.64
1,1,38,Ж,Санкт-Петербург,0.0,742601.31,0.0,0.0,166353.83,0.0,0.0,286133.38,114454.28,2020-01-09,Различные магазины,5921.0,722.30
2,1,38,Ж,Санкт-Петербург,0.0,742601.31,0.0,0.0,166353.83,0.0,0.0,286133.38,114454.28,2020-01-09,Розничные магазины,5411.0,769.27
3,1,38,Ж,Санкт-Петербург,0.0,742601.31,0.0,0.0,166353.83,0.0,0.0,286133.38,114454.28,2020-05-09,Розничные магазины,5411.0,5.90
4,1,38,Ж,Санкт-Петербург,0.0,742601.31,0.0,0.0,166353.83,0.0,0.0,286133.38,114454.28,2020-05-09,Личные услуги,7230.0,209.00


In [30]:
# Mapping gender_code to gnd_num
df_merged['gnd_num'] = df_merged['gender_code'].apply(lambda x: 2 if x == 'Ж' else 1 if x == 'М' else None)

In [31]:
# Dictionary to map the values
dir_mapping = {
    'Москва': 1,
    'Санкт-Петербург': 2,
    'Нижний Новгород': 3,
    'Казань': 4,
    'Томск': 5,
    'Ростов-на-Дону': 6,
    'Астрахань': 7,
    'Краснодар': 8,
    'Ставрополь': 9,
    'Тула': 10,
    'Воронеж': 11,
    'Новый Уренгой': 12,
    'Южно-Сахалинск': 13,
    'Югорск': 14,
    'Красноярск': 15,
    'Калининград': 16,
    'Пермь': 17,
    'Волгоград': 18,
    'ДРБЦР': 19,
    'Сургут': 20,
    'Екатеринбург': 21,
    'Благовещенск': 22,
    'Владивосток': 23,
    'Центральный': 24,
    'Новосибирск': 25,
    'Тюмень': 26,
    'Уфа': 27,
    'Самара': 28,
    'Кемерово': 29,
    'Якутск': 30,
    'Хабаровск': 31
}

# Mapping directory to dir_num
df_merged['dir_num'] = df_merged['directory'].map(dir_mapping)

In [32]:
# Dictionary to map the values
kind_mapping = {
    'Авиалинии, авиакомпании': 1,
    'Автомобили и транспортные средства': 2,
    'Аренда автомобилей': 3,
    'Бизнес услуги': 4,
    'Государственные услуги': 5,
    'Коммунальные и кабельные услуги': 6,
    'Контрактные услуги': 7,
    'Личные услуги': 8,
    'Магазины одежды': 9,
    'Оптовые поставщики и производители': 10,
    'Отели и мотели': 11,
    'Продажи по почте/телефону': 12,
    'Профессиональные услуги': 13,
    'Развлечения': 14,
    'Различные магазины': 15,
    'Розничные магазины': 16,
    'Строительные магазины': 17,
    'Транспортные услуги': 18,
    'Финансовые услуги': 19,
    'Членские взносы': 20,
}

# Map directory to dir_num
df_merged['trkind_num'] = df_merged['MCC_KIND_CD'].map(kind_mapping)

In [33]:
num_obsdt = df_merged.shape[0]

print(f"Number of Observations: {num_obsdt}")

df_merged.head(5)

Number of Observations: 77439


,client_id,age,gender_code,directory,aMRG_eop,aCSH_eop,aCRD_eop,pCUR_eop,pCRD_eop,pSAV_eop,pDEP_eop,sWork_S,tPOS_S,TRANSACTION_DT,MCC_KIND_CD,MCC_CD,CARD_AMOUNT_EQV_CBR,gnd_num,dir_num,trkind_num
0,1,38,Ж,Санкт-Петербург,0.0,742601.31,0.0,0.0,166353.83,0.0,0.0,286133.38,114454.28,2020-01-09,Розничные магазины,5411.0,51.64,2,2,16
1,1,38,Ж,Санкт-Петербург,0.0,742601.31,0.0,0.0,166353.83,0.0,0.0,286133.38,114454.28,2020-01-09,Различные магазины,5921.0,722.30,2,2,15
2,1,38,Ж,Санкт-Петербург,0.0,742601.31,0.0,0.0,166353.83,0.0,0.0,286133.38,114454.28,2020-01-09,Розничные магазины,5411.0,769.27,2,2,16
3,1,38,Ж,Санкт-Петербург,0.0,742601.31,0.0,0.0,166353.83,0.0,0.0,286133.38,114454.28,2020-05-09,Розничные магазины,5411.0,5.90,2,2,16
4,1,38,Ж,Санкт-Петербург,0.0,742601.31,0.0,0.0,166353.83,0.0,0.0,286133.38,114454.28,2020-05-09,Личные услуги,7230.0,209.00,2,2,8
